
### Modelo Hedônico IPTU — Treinamento Multifaixas com CatBoost

Este notebook realiza o treinamento de um **modelo hedônico** para estimar o valor de transação (ITBI/IPTU),
utilizando **CatBoostRegressor** com variáveis cadastrais do imóvel e **agregados mensais defasados (lag-1)**
por **setor fiscal** e **grupo de uso** (TERRENO vs EDIFICADO).

A abordagem é **multifaixas**: treina-se um modelo por combinação de:
- `grupo_uso` ∈ {`TERRENO`, `EDIFICADO`}
- `faixa` ∈ faixas de valor (<=500k, 500k–2mi, 2mi–10mi, >10mi)

A variável-alvo é modelada em **log** (`log1p(valor / escala)`) para estabilizar variância e melhorar ajuste.

%% [markdown]
### 1) Imports

- **glob / Path**: varredura recursiva dos Parquets na camada staging.
- **pandas / numpy**: manipulação de dados e engenharia de atributos.
- **catboost**: regressão com suporte nativo a variáveis categóricas.
- **sklearn**: validação cruzada (KFold) e métricas (MAE e R²).

In [2]:
!pip install -U scikit-learn catboost -q

In [2]:
import os
import glob
import json
import numpy as np
import pandas as pd
from pathlib import Path
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score


### 2) Configurações

- `STAGING_PATH`: caminho para os Parquets já padronizados (camada staging).
- `ARTEFATOS_PATH`: diretório para salvar modelos e metadados do treinamento.
- `TARGET_SCALE`: escala aplicada antes do log para melhorar estabilidade numérica.

#### Multifaixas
As faixas são definidas por `FAIXAS_BINS` e `FAIXAS_LABELS`, utilizadas para:
- segmentar o treinamento por nível de preço
- permitir modelos especializados em regimes distintos (ex.: imóveis baratos vs caros)

#### Variáveis categóricas
`CAT_COLS` são passadas explicitamente ao CatBoost via `cat_features` (sem necessidade de one-hot).

In [28]:
BRONZE_PATH = "../data/bronze/itbi"
ARTEFATOS_PATH = "../data/artefatos_modelo_multifaixas"
Path(ARTEFATOS_PATH).mkdir(exist_ok=True)

TARGET_SCALE = 1000.0

FAIXAS_BINS = [0, 500_000, 2_000_000, 10_000_000, np.inf]
FAIXAS_LABELS = [
    "faixa1_<=500k",
    "faixa2_500k_2mi",
    "faixa3_2mi_10mi",
    "faixa4_>10mi"
]

CAT_COLS = [
    "setor_fiscal",
    "descricao_uso_iptu",
    "descricao_padrao_iptu"
]

### 3) Carregar Parquets (camada staging)

- Varre recursivamente todos os arquivos `.parquet` em `STAGING_PATH`.
- Concatena em um único DataFrame para treinamento.

In [5]:
print(">> Carregando parquets...")
files = glob.glob(f"{BRONZE_PATH}/**/*.parquet", recursive=True)
df = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
print("Base carregada:", len(df), "linhas")


>> Carregando parquets...
Base carregada: 2600551 linhas


### 4) Filtros de qualidade e recorte do domínio

Objetivo dos filtros:
- restringir para transações comparáveis ao fenômeno modelado (compra e venda)
- evitar registros com status não elegíveis
- reduzir ruído extremo por outliers de valor

Regras aplicadas:
- `natureza_transacao = "1.Compra e venda"`
- `sql_status em "Ativo Territorial"`
- `valor_transacao_declarado entre 5000 a 50000000`
- `proporcao_transmitida_percent igual a 100%`
- `situacao_sql = "Ativo Territorial"`

In [ ]:
df = df[df["natureza_transacao"] == "1.Compra e venda"]

df = df[df["situacao_sql"].isin(["Ativo Predial", "Ativo Territorial"])]

df = df[df["valor_transacao_declarado"].between(5000, 50_000_000)]

df = df[df["proporcao_transmitida_percent"] == 100]

df = df[df["situacao_sql"] == "Ativo Predial"]


### 5) Feature engineering (atributos temporais e do imóvel)

Atributos criados:
- `year`, `month`, `yyyymm`: granularidade temporal e chave mensal
- `imovel_idade`: idade aproximada do imóvel (ano transação - ano conclusão)
- `grupo_uso`: binarização simples de uso: TERRENO vs EDIFICADO

**Nota:** `imovel_idade` é truncada em 0 para evitar negativos (dados inconsistentes).
Valores ausentes são tratados como 0 (estratégia conservadora).

In [ ]:
df["data_transacao"] = pd.to_datetime(df["data_transacao"], errors="coerce")
df["year"] = df["data_transacao"].dt.year
df["month"] = df["data_transacao"].dt.month
df["yyyymm"] = df["year"] * 100 + df["month"]

# Idade do imóvel (ano de transação - ano de conclusão)
df["imovel_idade"] = df["year"] - df["ano_conclusao_construcao_iptu"]
df["imovel_idade"] = df["imovel_idade"].clip(lower=0).fillna(0)

# Grupo de uso (heurística por descrição)
df["grupo_uso"] = np.where(
    df["descricao_uso_iptu"].str.contains("terren", case=False, na=False),
    "TERRENO",
    "EDIFICADO"
)

### 6) Agregados mensais por setor fiscal (com defasagem)

Nesta etapa criamos variáveis de contexto de mercado, por **mês**, **setor fiscal** e **grupo de uso**.

Métricas mensais:
- `median_total`: mediana do valor declarado no grupo
- `median_sqm`: (atualmente igual à mediana do total; ver observação abaixo)
- `count`: quantidade de transações no mês (proxy de liquidez)

Em seguida, calculamos `lag-1` (mês anterior) para evitar vazamento temporal:
- `median_total_lag1`, `median_sqm_lag1`, `count_lag1`

**Observação importante:** `median_sqm` está calculado com a mesma expressão de `median_total`.
Se a intenção for mediana do preço por m², o correto seria:
`declared_transaction_value / built_area_sqm` (ou outra área relevante), com tratamento de divisão por zero.

In [ ]:

monthly = (
    df.groupby(["grupo_uso", "setor_fiscal", "yyyymm"])
      .agg(
          median_total=("valor_transacao_declarado", "median"),
          median_sqm=("valor_transacao_declarado", lambda x: np.median(x)),
          count=("valor_transacao_declarado", "count")
      )
      .reset_index()
)

monthly["median_total_lag1"] = (
    monthly.sort_values("yyyymm")
           .groupby(["grupo_uso", "setor_fiscal"])["median_total"]
           .shift(1)
)

monthly["median_sqm_lag1"] = (
    monthly.sort_values("yyyymm")
           .groupby(["grupo_uso", "setor_fiscal"])["median_sqm"]
           .shift(1)
)

monthly["count_lag1"] = (
    monthly.sort_values("yyyymm")
           .groupby(["grupo_uso", "setor_fiscal"])["count"]
           .shift(1)
)

df = df.merge(
    monthly[[
        "grupo_uso",
        "setor_fiscal",
        "yyyymm",
        "median_total_lag1",
        "median_sqm_lag1",
        "count_lag1"
    ]],
    on=["grupo_uso", "setor_fiscal", "yyyymm"],
    how="left"
)

### 7) Definição do target (log-transform)

Para reduzir assimetria (cauda longa) típica de valores imobiliários:
- Definimos `target = log1p(valor / TARGET_SCALE)`

No momento da avaliação (predição), desfazemos a transformação:
- `valor_pred = expm1(pred) * TARGET_SCALE`

In [ ]:
df["target"] = np.log1p(df["valor_transacao_declarado"] / TARGET_SCALE)

### 8) Criação das faixas de valor

Segmenta cada registro em uma faixa baseada no `valor_transacao_declarado`.
Isso permite treinar modelos especializados por regime de preços.

In [ ]:
df["faixa"] = pd.cut(
    df["valor_transacao_declarado"],
    bins=FAIXAS_BINS,
    labels=FAIXAS_LABELS
)

### 9) Treinamento dos modelos (KFold 5x)

Para cada `grupo_uso` e `faixa`:
- Filtra o subconjunto (`sub`)
- Se houver menos de 1000 registros, pula (amostra insuficiente)
- Cria matriz `X` com:
  - Categóricas: `CAT_COLS`
  - Numéricas: área construída, área do terreno, testada, idade, yyyymm, lags do setor
- Realiza validação cruzada KFold (5 folds)
- Métricas por fold:
  - **MAE** em R$ (no espaço original, após inversão do log)
  - **R²** (coeficiente de determinação)

Ao final, salva o último modelo treinado da iteração em:
`artefatos_modelo_multifaixas/modelo_<grupo>_<faixa>.cbm`

In [24]:
modelos = {}

for grupo in df["grupo_uso"].unique():
    modelos[grupo] = {}

    for faixa in FAIXAS_LABELS:
        sub = df[(df["grupo_uso"] == grupo) & (df["faixa"] == faixa)]

        # Evita treinar com pouca amostra
        if len(sub) < 1000:
            continue

        print(f"\nTreinando {grupo} - {faixa} ({len(sub)} linhas)")

        X = sub[CAT_COLS + [
            "area_construida_m2",
            "area_terreno_m2",
            "testada_m",
            "imovel_idade",
            "yyyymm",
            "median_total_lag1",
            "median_sqm_lag1",
            "count_lag1"
        ]]

        y = sub["target"]

        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        maes = []
        r2s = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = CatBoostRegressor(
                iterations=400,
                depth=6,
                learning_rate=0.05,
                loss_function="RMSE",
                verbose=False
            )

            model.fit(
                Pool(X_train, y_train, cat_features=CAT_COLS),
                eval_set=Pool(X_val, y_val, cat_features=CAT_COLS),
                verbose=False
            )

            # volta do log para R$
            pred = np.expm1(model.predict(X_val)) * TARGET_SCALE
            true = np.expm1(y_val) * TARGET_SCALE

            mae = mean_absolute_error(true, pred)
            r2 = r2_score(true, pred)

            maes.append(mae)
            r2s.append(r2)

            print(f"[Fold {fold}] MAE: R$ {mae:,.0f} | R²: {r2:.3f}")

        print(f">> MÉDIA MAE: R$ {np.mean(maes):,.0f} | R²: {np.mean(r2s):.3f}")

        # guarda e salva o último modelo treinado
        modelos[grupo][faixa] = model

        model.save_model(
            f"{ARTEFATOS_PATH}/modelo_{grupo}_{faixa}.cbm"
        )



Treinando EDIFICADO - faixa1_<=500k (1219963 linhas)


KeyError: "['fiscal_sector', 'usage_description_iptu', 'construction_standard_description_iptu'] not in index"